In [1]:
import pandas as pd

# Load training and test data
train = pd.read_csv("../../DSML 27/Loan_System/test_Y3wMUE5_7gLdaTN.csv")
test = pd.read_csv("../../DSML 27/Loan_System/train_u6lujuX_CVtuZ9i.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()


Train shape: (367, 12)
Test shape: (614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [3]:

import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load training and test data (corrected file paths)
train = pd.read_csv("../../DSML 27/Loan_System/train_u6lujuX_CVtuZ9i.csv")
test = pd.read_csv("../../DSML 27/Loan_System/test_Y3wMUE5_7gLdaTN.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)

# Safe drop Loan_ID if it exists
if "Loan_ID" in train.columns:
    train.drop("Loan_ID", axis=1, inplace=True)

if "Loan_ID" in test.columns:
    test_IDs = test["Loan_ID"]  # Save for submission
    test.drop("Loan_ID", axis=1, inplace=True)
else:
    test_IDs = pd.Series(["Unknown_ID_" + str(i) for i in range(len(test))])

# Combine train & test (excluding Loan_Status if it exists)
if "Loan_Status" in train.columns:
    combined = pd.concat([train.drop("Loan_Status", axis=1), test], axis=0)
else:
    combined = pd.concat([train, test], axis=0)

# Fill missing values
combined["Gender"].fillna(combined["Gender"].mode()[0], inplace=True)
combined["Married"].fillna(combined["Married"].mode()[0], inplace=True)
combined["Dependents"].fillna(combined["Dependents"].mode()[0], inplace=True)
combined["Self_Employed"].fillna(combined["Self_Employed"].mode()[0], inplace=True)
combined["Credit_History"].fillna(combined["Credit_History"].mode()[0], inplace=True)
combined["Loan_Amount_Term"].fillna(combined["Loan_Amount_Term"].mode()[0], inplace=True)
combined["LoanAmount"].fillna(combined["LoanAmount"].median(), inplace=True)

# Encode categorical features
cat_cols = ["Gender", "Married", "Dependents", "Education", "Self_Employed", "Property_Area"]
le = LabelEncoder()
for col in cat_cols:
    combined[col] = le.fit_transform(combined[col])

# Split combined back into training and test sets
X = combined.iloc[:train.shape[0], :]
X_test = combined.iloc[train.shape[0]:, :]

# Prepare target variable if it exists
if "Loan_Status" in train.columns:
    y = train["Loan_Status"].map({"Y": 1, "N": 0})
else:
    y = None

Train shape: (614, 13)
Test shape: (367, 12)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9088\1498754221.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined["Gender"].fillna(combined["Gender"].mode()[0], inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9088\1498754221.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Split train set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Validation Predictions
y_pred = model.predict(X_val)

# Evaluation
print("Accuracy:", accuracy_score(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))


Accuracy: 0.7560975609756098
[[18 25]
 [ 5 75]]
              precision    recall  f1-score   support

           0       0.78      0.42      0.55        43
           1       0.75      0.94      0.83        80

    accuracy                           0.76       123
   macro avg       0.77      0.68      0.69       123
weighted avg       0.76      0.76      0.73       123



In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB()
}

for name, clf in models.items():
    clf.fit(X_train, y_train)
    pred = clf.predict(X_val)
    print(f"\n{name}")
    print("Accuracy:", accuracy_score(y_val, pred))



Logistic Regression
Accuracy: 0.7886178861788617

Decision Tree
Accuracy: 0.6829268292682927

SVM
Accuracy: 0.6504065040650406

KNN
Accuracy: 0.5772357723577236

Naive Bayes
Accuracy: 0.7886178861788617


c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ✅ Predict using trained model on test data
y_pred_test = model.predict(X_test)

# �� Show predictions (since we don't have true labels for test data)
print("✅ Predictions on test data:")
print(f"Total test samples: {len(y_pred_test)}")
print(f"Predicted 'Approved' (1): {sum(y_pred_test == 1)}")
print(f"Predicted 'Rejected' (0): {sum(y_pred_test == 0)}")
print(f"Approval rate: {sum(y_pred_test == 1) / len(y_pred_test):.2%}")

# 📋 Create submission dataframe
submission = pd.DataFrame({
    'Loan_ID': test_IDs,
    'Loan_Status': ['Y' if pred == 1 else 'N' for pred in y_pred_test]
})

print("\n�� Sample predictions:")
print(submission.head(10))

# �� Save predictions (optional)
# submission.to_csv('loan_predictions.csv', index=False)
# print("\n💾 Predictions saved to 'loan_predictions.csv'")



✅ Predictions on test data:
Total test samples: 367
Predicted 'Approved' (1): 293
Predicted 'Rejected' (0): 74
Approval rate: 79.84%

�� Sample predictions:
    Loan_ID Loan_Status
0  LP001015           Y
1  LP001022           Y
2  LP001031           Y
3  LP001035           Y
4  LP001051           N
5  LP001054           Y
6  LP001055           Y
7  LP001056           N
8  LP001059           Y
9  LP001067           Y


In [13]:
import pickle

# Save the trained model
with open('loan_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("✅ Model saved as 'loan_model.pkl'")

# Optional: Save the LabelEncoder as well (for preprocessing new data)
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(le, file)

print("✅ LabelEncoder saved as 'label_encoder.pkl'")

# Optional: Save the test IDs for future reference
with open('test_ids.pkl', 'wb') as file:
    pickle.dump(test_IDs, file)

print("✅ Test IDs saved as 'test_ids.pkl'")

✅ Model saved as 'loan_model.pkl'
✅ LabelEncoder saved as 'label_encoder.pkl'
✅ Test IDs saved as 'test_ids.pkl'
